In [1]:
import numpy as np
import datetime
from datetime import date, timedelta
import pandas as pd
import jaydebeapi
from pyhive import presto
import trino


bifrost_user = "karan.singhal@myntra.com"
bifrost_password = "49802e6e7b895d3cf9ab51cb31b5afb15570e629b4162417b76bbffc6f54ef36b5e53620c2279f66687cb3a6840d7766e93757730900d9aa9097e0095c9ddc72"
bifrost_link = "jdbc:bifrost://bifrostx-gateway.myntra.com:80/hive"

 

#Executable Cursor
conn = jaydebeapi.connect("com.myntra.bifrost.jdbc.BifrostDriver",
                          bifrost_link,
                          [bifrost_user, bifrost_password],
                          "C:\\Users\\karan.singhal\\Downloads\\bifrost-jdbc-3.0-SNAPSHOT-ALPHA1.jar")

cur = conn.cursor()
print("Connection made ✅ ")

Connection made ✅ 


In [2]:
import pandas as pd
import numpy as np
import yagmail
from sqlalchemy import create_engine,text
import time
from datetime import datetime
import os
import pylab
import jaydebeapi
import requests
import gzip
import urllib
from urllib import request
import base64
import traceback
import string
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

## Bifrost Creds
bifrost_class = "com.myntra.bifrost.jdbc.BifrostDriver"
bifrost_url = "jdbc:bifrost://bifrostx-gateway.myntra.com:80/delta_prod"


bifrost_user = "karan.singhal@myntra.com"
bifrost_password="d096713e0d0b164a95adb35493997680467c9586607c8729e27a15deafb9027f1d4c84d87671c6ce8f21410c52019cb7e1358ebc336b9da836dfc807d4ab269f"


jdbc_path = "bifrost-jdbc-3.0-SNAPSHOT-ALPHA1.jar"


def fetch_data_from_bifrost(query, path=None, filename=None,return_result = True):
    
#     temp_dir = 'C:\\Users\\300066847\\Desktop\\PyCodes\\dump\\'
    temp_dir = 'C:\\Users\\karan.singhal\\Downloads\\'
    try:
        query_submit_url = "http://bifrostx-gateway.myntra.com/api/v1/client/async/queries"  
        auth_token = base64.b64encode(bytes(bifrost_user + ":" + bifrost_password,'UTF-8')).decode('UTF-8')
        query = query.replace("\n","\\n")

        payload = """{\"query\":\"{query}\", \"engine\":\"PRESTO\", \"username\": \"{username}\", \"output\": {\"format\":
            \"{output_format}\", \"compression\": \"{output_compression}\"}} """
        payload = payload.replace("{query}", query).replace("{username}", bifrost_user).replace("{output_format}",
                                                                                                "CSV").replace(
                "{output_compression}", "GZIP")
        headers = {
                'Authorization': '{auth_token}'.replace("{auth_token}", auth_token),
                'Content-Type': 'application/json'
            }

        query_submit_response = requests.request("POST", query_submit_url, headers = headers, data = payload  )
        query_submit_response_json = query_submit_response.json()
        #print(query_submit_response_json)
        if query_submit_response_json['token'] == 'None':
            raise Exception(query_submit_response_json['description'])
        query_status_url = query_submit_response_json["_links"]["status"]["href"] 

        while True:
            query_status_response = requests.request(method="GET", url = query_status_url, 
                                                     headers = {'Id-token': 'secret'}, data = "" )
            if query_status_response is not None and query_status_response.json()["status"] == "COMPLETED":
                query_status_response_json = query_status_response.json()

                file_path = query_status_response_json["externalLocation"]

                ## Unzipping file
                with urllib.request.urlopen(file_path) as response:
                    with gzip.GzipFile(fileobj=response) as uncompressed:
                        file_content =  uncompressed.read()

                token = query_submit_response_json["token"]
                
                #File descriptor
                filename = filename if filename else token
                path = path if path else temp_dir
                dest_file = path+filename+".csv"

                with open(dest_file, 'wb') as f:
                    f.write(file_content)
                break

            elif query_status_response is not None and query_status_response.json()["status"] == "FAILED":
                print("Failed to execute query..!! ")
                raise Exception(query_status_response.json()["reason"])

            else:
                time.sleep(2)
        if return_result == True:
            df = pd.read_csv(dest_file)
            os.remove(dest_file)
            return df

    except Exception as e:
        print(e)
        traceback.print_exc()

In [3]:
import pygsheets
import pandas as pd
import traceback

def fetching_data_from_gsheet(gsheet_url, worksheet_title):
    # Authorize using the service account credentials
    gc = pygsheets.authorize(service_file=r'C:\Users\karan.singhal\Downloads\AccessKey.json')
    
    # Open the Google Sheet by URL
    data = gc.open_by_url(gsheet_url)
    
    # Select the specified worksheet
    hop = data.worksheet('title', worksheet_title)
    
    # Get the data as a DataFrame
    raw_data = hop.get_as_df()
    
    return raw_data

def get_googleworkbook_sheet_names(workbook):
    return [ws.title for ws in workbook.worksheets()]

def write_to_googlesheet(data, gsheet_url, sheetname):
    try:
        # Authorize using the service account credentials
        gc = pygsheets.authorize(service_file=r'C:\Users\karan.singhal\Downloads\AccessKey.json')
        
        # Open the Google Sheet by URL
        workbook = gc.open_by_url(gsheet_url)
        
        # Prepare the data
        data.fillna(0, inplace=True)
        all_sheets = [s.title for s in workbook.worksheets()]

        if sheetname in all_sheets:
            my_sheet = workbook.worksheet('title', sheetname)
            # Clear only columns A to H
            my_sheet.clear(start='A1', end='K' + str(my_sheet.rows))
        else:
            print(f"Couldn't find {sheetname}, creating a new sheet.")
            my_sheet = workbook.add_worksheet(title=sheetname, rows=data.shape[0], cols=data.shape[1])
        
        # Update the sheet with new data
        my_sheet.set_dataframe(data, (1, 1))  # Starting from the first cell (A1)

    except Exception as error:
        print(f"An error occurred while writing to Google Sheet {sheetname}: {error}")
        traceback.print_exc()

# Example usage
gsheet_url = 'https://docs.google.com/spreadsheets/d/1Dp3wO0LqH0JiriKOlRXJ7s8st1BZ7ObX3zSoa6ZdQjI/edit?usp=sharing'

In [4]:
df = fetching_data_from_gsheet(gsheet_url, 'BPC M-Now P0 Styles')
df.shape

C:\Users\karan.singhal\AppData\Roaming\Python\Python312\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


(5418, 72)

In [5]:
df.columns = df.iloc[2]  
df = df[3:].reset_index(drop=True)  
df

2,Style ID,Article Type,Brand,Brand House,CM,ROS,Followup-poc,36,309,28,...,,,,,,,,1225 styles,Inactive,Remarks
0,29204096,Toothpaste and Tooth Powder,ZANDU,Emami,Chaithra,28.9,Vinay,176,480,0,...,,,,,,,,#N/A,#N/A,Brand yet to revert
1,22844406,Cologne and After Shave,YARDLEY LONDON,Wipro,Gurneet,1.7,Vinay,0,0,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
2,22292788,Body Mist and Spray,YARDLEY LONDON,Wipro,Gurneet,2.5,Vinay,48,48,0,...,,,,,,,,#N/A,#N/A,Brand yet to revert
3,22292786,Roll-Ons,YARDLEY LONDON,Wipro,Gurneet,6.5,Vinay,48,144,0,...,,,,,,,,#N/A,#N/A,Brand yet to revert
4,22292780,Roll-Ons,YARDLEY LONDON,Wipro,Gurneet,6.2,Vinay,96,96,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5410,27615368,Deodorant,ADIDAS,DELIVERYBOX,Prasad,30.2,Kiran,0,0,39,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
5411,27615364,Deodorant,ADIDAS,DELIVERYBOX,Prasad,5.9,Kiran,0,34,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
5412,27615354,Perfume,ADIDAS,DELIVERYBOX,Prasad,1.7,Kiran,0,15,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
5413,27615352,Perfume,ADIDAS,DELIVERYBOX,Prasad,3.2,Kiran,0,0,1,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted


In [6]:
df1 = df.iloc[:, [0] + list(range(7, df.shape[1]))]
df1

2,Style ID,36,309,28,15774,81,16228,36,309,28,...,,,,,,,,1225 styles,Inactive,Remarks
0,29204096,176,480,0,272,304,1232,38122,103968,0,...,,,,,,,,#N/A,#N/A,Brand yet to revert
1,22844406,0,0,0,0,48,48,0,0,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
2,22292788,48,48,0,0,0,96,7068,7068,0,...,,,,,,,,#N/A,#N/A,Brand yet to revert
3,22292786,48,144,0,96,96,384,7171,21514,0,...,,,,,,,,#N/A,#N/A,Brand yet to revert
4,22292780,96,96,0,96,96,384,14342,14342,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5410,27615368,0,0,39,289,0,328,0,0,10617,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
5411,27615364,0,34,0,57,70,161,0,8786,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
5412,27615354,0,15,0,0,26,41,0,16648,0,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted
5413,27615352,0,0,1,45,99,145,0,0,1110,...,,,,,,,,#N/A,#N/A,No Visibility shared but slotted


In [7]:
id_col = ["Style ID"]  
wh_id_col = [36, 309, 28, 15774, 81]

group1_cols = df1.iloc[:, [0] + list(range(1, 6))]   # Include Col 1 + Open PO (Cols 2-6)
group2_cols = df1.iloc[:, [0] + list(range(13, 18))]  # Include Col 1 + Dispatched qty-1 (Skip Col 7)
group3_cols = df1.iloc[:, [0] + list(range(19, 24))] # Include Col 1 + Dispatched qty-2 (Skip Col 13)
group4_cols = df1.iloc[:, [0] + list(range(25, 30))] # Include Col 1 + Dispatch Date-1 (Skip Col 19)
group5_cols = df1.iloc[:, [0] + list(range(30, 35))] # Include Col 1 + Dispatch Date-2 (Skip Col 25)


df_pivot1 = pd.melt(group1_cols, id_vars= id_col , value_vars= wh_id_col , var_name='Wh_ID', value_name='Open PO')
df_pivot2 = pd.melt(group2_cols, id_vars= id_col , value_vars= wh_id_col , var_name='Wh_ID', value_name='Dispatched qty-1')
df_pivot3 = pd.melt(group3_cols, id_vars= id_col , value_vars= wh_id_col , var_name='Wh_ID', value_name="Dispatched qty-2")
df_pivot4 = pd.melt(group4_cols, id_vars= id_col , value_vars= wh_id_col , var_name='Wh_ID', value_name="Dispatch Date-1")
df_pivot5 = pd.melt(group5_cols, id_vars= id_col , value_vars= wh_id_col , var_name='Wh_ID', value_name="Dispatch Date-2")

# Merge all pivots on "Style ID" and "Warehouse_ID"
df_final_qc = (
    df_pivot1
    .merge(df_pivot2, on=["Style ID", "Wh_ID"])
    .merge(df_pivot3, on=["Style ID", "Wh_ID"])
    .merge(df_pivot4, on=["Style ID", "Wh_ID"])
    .merge(df_pivot5, on=["Style ID", "Wh_ID"])
)

df_final_qc

,Style ID,Wh_ID,Open PO,Dispatched qty-1,Dispatched qty-2,Dispatch Date-1,Dispatch Date-2
0,29204096,36,176,Brand Yet To Revert,,,
1,22844406,36,0,No Open Po,,,
2,22292788,36,48,Brand Yet To Revert,,,
3,22292786,36,48,Brand Yet To Revert,,,
4,22292780,36,96,Supply Issue,,,
...,...,...,...,...,...,...,...
27070,27615368,81,0,No Open Po,,,
27071,27615364,81,70,Supply Issue,,,
27072,27615354,81,26,Supply Issue,,,
27073,27615352,81,99,Supply Issue,,,


In [8]:
df_final_qc = df_final_qc.replace(['No Open PO', np.nan, '', 'No Open Po'], 0)

C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\2790512985.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_qc = df_final_qc.replace(['No Open PO', np.nan, '', 'No Open Po'], 0)


In [9]:
zero_dispatched = (df_final_qc[['Dispatched qty-1', 'Dispatched qty-2']] == 0).any(axis=1)
non_zero_dates = (df_final_qc[['Dispatch Date-1', 'Dispatch Date-2']]!= 0).all(axis=1)

if zero_dispatched.any() and non_zero_dates.any():  # Both conditions must be True
    print('Failed QCs')
else:
    print('All QCs passed')

Failed QCs


In [10]:
non_zero_dates

0        False
1        False
2        False
3        False
4        False
         ...  
27070    False
27071    False
27072    False
27073    False
27074    False
Length: 27075, dtype: bool

In [11]:
print(df_final_qc.dtypes)

Style ID             int64
Wh_ID                int64
Open PO              int64
Dispatched qty-1    object
Dispatched qty-2    object
Dispatch Date-1     object
Dispatch Date-2     object
dtype: object


In [12]:
# Remove any non-numeric characters (except dots) from 'Dispatched qty-1' and 'Dispatched qty-2'
df_final_qc['Dispatched qty-1'] = df_final_qc['Dispatched qty-1'].replace(r'[^0-9.]', '', regex=True)
df_final_qc['Dispatched qty-2'] = df_final_qc['Dispatched qty-2'].replace(r'[^0-9.]', '', regex=True)

# Convert 'Open PO', 'Dispatched qty-1', and 'Dispatched qty-2' to numeric (float), coercing errors if any
df_final_qc['Open PO'] = pd.to_numeric(df_final_qc['Open PO'], errors='coerce').astype(float)
df_final_qc['Dispatched qty-1'] = pd.to_numeric(df_final_qc['Dispatched qty-1'], errors='coerce').astype(float)
df_final_qc['Dispatched qty-2'] = pd.to_numeric(df_final_qc['Dispatched qty-2'], errors='coerce').astype(float)

In [13]:
import smtplib
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import sys

df_final_qc = pd.DataFrame(df_final_qc)



df_final_qc['Dispatch Date-1'] = pd.to_datetime(df_final_qc['Dispatch Date-1'], errors='coerce')
df_final_qc['Dispatch Date-2'] = pd.to_datetime(df_final_qc['Dispatch Date-2'], errors='coerce')

# Replace NaN values with 0 if necessary
df_final_qc.fillna(0, inplace=True)

# List to store failed QC messages
failed_qc_messages = []

# QC 1: Check if any dispatched quantity is zero while dispatch dates are valid
zero_dispatched = (df_final_qc[['Dispatched qty-1', 'Dispatched qty-2']] == 0).any(axis=1)
non_zero_dates = (df_final_qc['Dispatch Date-1'] != "1970-01-01") & (df_final_qc['Dispatch Date-2'] != "1970-01-01")
qc1_failed_rows = df_final_qc[zero_dispatched & non_zero_dates]

if not qc1_failed_rows.empty:
    failed_qc_messages.append("<b>QC 1 Failure:</b> Some dispatches have zero quantity despite valid dispatch dates.<br><pre>" +
                              qc1_failed_rows.to_html(index=False) + "</pre>")

# QC 2: Check if Dispatched qty-1 is greater than Open PO
qc2_failed_rows = df_final_qc[df_final_qc['Open PO'] < df_final_qc['Dispatched qty-1']]
if not qc2_failed_rows.empty:
    failed_qc_messages.append("<b>QC 2 Failure:</b> Dispatched qty-1 is greater than Open PO.<br><pre>" +
                              qc2_failed_rows.to_html(index=False) + "</pre>")

# QC 3: Check if Dispatched qty-2 is greater than Open PO
qc3_failed_rows = df_final_qc[df_final_qc['Open PO'] < df_final_qc['Dispatched qty-2']]
if not qc3_failed_rows.empty:
    failed_qc_messages.append("<b>QC 3 Failure:</b> Dispatched qty-2 is greater than Open PO.<br><pre>" +
                              qc3_failed_rows.to_html(index=False) + "</pre>")

# QC 4: Check if total dispatched quantity exceeds Open PO
qc4_failed_rows = df_final_qc[df_final_qc['Open PO'] < (df_final_qc['Dispatched qty-1'] + df_final_qc['Dispatched qty-2'])]
if not qc4_failed_rows.empty:
    failed_qc_messages.append("<b>QC 4 Failure:</b> Dispatched qty-1 + Dispatched qty-2 is greater than Open PO.<br><pre>" +
                              qc4_failed_rows.to_html(index=False) + "</pre>")

# QC 5: Check if Dispatch Date-1 is equal to Dispatch Date-2 and both Dispatched qty-1 and Dispatched qty-2 are not equal to 0
qc5_failed_rows = df_final_qc[(df_final_qc['Dispatch Date-1'] == df_final_qc['Dispatch Date-2']) & 
                              (df_final_qc['Dispatched qty-1'] != 0) &
                              (df_final_qc['Dispatched qty-2'] != 0)]

if not qc5_failed_rows.empty:
    failed_qc_messages.append("<b>QC 5 Failure:</b> Dispatch Date-1 is equal to Dispatch Date-2 and both Dispatched qty-1 and Dispatched qty-2 are not 0.<br><pre>" +
                              qc5_failed_rows.to_html(index=False) + "</pre>")


# If any QC fails, send an email with only the failed messages
if failed_qc_messages:
    print("QC Failures Detected")

    # Email Configuration
    sender_email = "karan.singhal@myntra.com"
    receiver_emails = [
        "mayank.jain4@myntra.com", "shobendra.jain1@myntra.com",
        "karan.singhal@myntra.com", "mannem.teja@myntra.com",
        "saravanan.palani@myntra.com", 'deepikanaik.vc@myntra.com', 'adityakumarkt.vc@myntra.com', 'sushmithab.vc@myntra.com', 'gagans.vc@myntra.com'
    ]
    subject = "QC Alert: QC Failures Detected"

    # Start of the email body
    body = """
    <p>Hi Team,</p>
    <p>There is a QC failure detected. <a href="mailto:deepikanaik.vc@myntra.com"><strong>@Deepika Naik</strong></a>, please look into it.</p>
    <p>The details of the failed QC checks are as follows:</p>
    <br><br>
    
    """

    # Append all QC failure messages
    body += "<br><br>".join(failed_qc_messages)

    # Adding Best Regards and Karan's name
    body += """
    <br><br>
    <p>Kindly revert back on this email once the data is corrected</p>
    <p>Best Regards,</p>
    <p>Karan Singhal</p>
    """

    # Email Setup
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(receiver_emails)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))  # Send as HTML for bold formatting

    # Sending Email
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, "ziis qwbo uicl bgia")  # App password or replace with a secure method
        server.sendmail(sender_email, receiver_emails, msg.as_string())
        server.quit()
        print("Email sent successfully to all recipients!")
    except Exception as e:
        print(f"Error sending email: {e}")

    sys.exit(1)
else:
    print("All QCs passed successfully")

All QCs passed successfully


In [14]:
df_final_qc['Dispatched qty-1'] = pd.to_numeric(df_final_qc['Dispatched qty-1'], errors='coerce')

In [15]:
# Merge all pivots on "Style ID" and "Warehouse_ID"
df_final = (
    df_pivot1
    .merge(df_pivot2, on=["Style ID", "Wh_ID"])
    .merge(df_pivot4, on=["Style ID", "Wh_ID"])
)

df_final

,Style ID,Wh_ID,Open PO,Dispatched qty-1,Dispatch Date-1
0,29204096,36,176,Brand Yet To Revert,
1,22844406,36,0,No Open Po,
2,22292788,36,48,Brand Yet To Revert,
3,22292786,36,48,Brand Yet To Revert,
4,22292780,36,96,Supply Issue,
...,...,...,...,...,...
27070,27615368,81,0,No Open Po,
27071,27615364,81,70,Supply Issue,
27072,27615354,81,26,Supply Issue,
27073,27615352,81,99,Supply Issue,


In [16]:
# Merge all pivots on "Style ID" and "Warehouse_ID"
df_final1 = (
    df_pivot1
    .merge(df_pivot3, on=["Style ID", "Wh_ID"])
    .merge(df_pivot5, on=["Style ID", "Wh_ID"])
)

df_final1

,Style ID,Wh_ID,Open PO,Dispatched qty-2,Dispatch Date-2
0,29204096,36,176,,
1,22844406,36,0,,
2,22292788,36,48,,
3,22292786,36,48,,
4,22292780,36,96,,
...,...,...,...,...,...
27070,27615368,81,0,,
27071,27615364,81,70,,
27072,27615354,81,26,,
27073,27615352,81,99,,


In [17]:
df_final1["Dispatched qty-2"] = pd.to_numeric(df_final1["Dispatched qty-2"], errors="coerce")
df_final2 = df_final1[df_final1["Dispatched qty-2"] > 0]
df_final2["Dispatched qty-2"] = pd.to_numeric(df_final2["Dispatched qty-2"], errors="coerce").fillna(0).astype(int)
df_final2

C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\4184305284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2["Dispatched qty-2"] = pd.to_numeric(df_final2["Dispatched qty-2"], errors="coerce").fillna(0).astype(int)


,Style ID,Wh_ID,Open PO,Dispatched qty-2,Dispatch Date-2
314,2212867,36,7560,6120,28/03/25
8996,32062431,309,200,50,24/03/25
8997,32062430,309,150,50,24/03/25
9051,24877096,309,140,80,24/03/25
9052,24877094,309,220,140,24/03/25
9054,24830798,309,561,228,24/03/25
9061,23540690,309,390,90,24/03/25
9062,23540688,309,1248,144,24/03/25
9063,23540686,309,224,32,24/03/25
9064,23540684,309,168,24,24/03/25


In [18]:
df_final.rename(columns={'Dispatched qty-1': 'Dispatched qty', 'Dispatch Date-1': 'Dispatch Date' }, inplace=True) 
df_final2.rename(columns={'Dispatched qty-2': 'Dispatched qty', 'Dispatch Date-2': 'Dispatch Date' }, inplace=True)

C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\1403934983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2.rename(columns={'Dispatched qty-2': 'Dispatched qty', 'Dispatch Date-2': 'Dispatch Date' }, inplace=True)


In [19]:
# List of DataFrames to concatenate
dfs = [df_final, df_final2]

# Concatenate the DataFrames vertically (row-wise)
df_combined = pd.concat(dfs, ignore_index=True)  # ignore_index to reset the index
df_combined

,Style ID,Wh_ID,Open PO,Dispatched qty,Dispatch Date
0,29204096,36,176,Brand Yet To Revert,
1,22844406,36,0,No Open Po,
2,22292788,36,48,Brand Yet To Revert,
3,22292786,36,48,Brand Yet To Revert,
4,22292780,36,96,Supply Issue,
...,...,...,...,...,...
27109,23540690,81,30,30,20/03/25
27110,23540688,81,720,192,20/03/25
27111,23540686,81,32,32,20/03/25
27112,23540684,81,25,24,20/03/25


In [20]:
style_id = tuple(map(str, df_combined['Style ID'].unique()))

In [21]:
df_combined['BU'] = 'Personal Care'

In [22]:
po_value = fetch_data_from_bifrost('''
with landed_price as (select dp.style_id , po.* ,
row_number () over(partition by po.sku_id order by po.created_date DESC) AS rn  from bidb.fact_purchase_order po
left join bidb.dim_product dp on dp.sku_id = po.sku_id
where created_date is not null and dp.style_id in {0})

select distinct style_id, item_landed_price from landed_price where rn=1
'''.format(style_id))

In [23]:
df_combined1 = pd.merge(df_combined,po_value,left_on="Style ID",right_on="style_id",how="left")

In [24]:
df_combined1['Remarks'] = df_combined1['Dispatched qty']
df_combined1['Remarks'] = df_combined1['Remarks'].apply(lambda x: 'Slotted' if str(x).isdigit() else x)
df_combined1['Remarks'] = df_combined1['Remarks'].fillna('Others')

df_combined1['Dispatched qty'] = pd.to_numeric(df_combined1['Dispatched qty'], errors='coerce').fillna(0)

In [25]:
df_combined1['Dispatched qty'] = pd.to_numeric(df_combined1['Dispatched qty'], errors='coerce')
df_combined1['item_landed_price'] = pd.to_numeric(df_combined1['item_landed_price'], errors='coerce')


df_combined1['Dispatched qty'].fillna(0, inplace=True)
df_combined1['item_landed_price'].fillna(0, inplace=True)


df_combined1['Dispatch Value'] = df_combined1['Dispatched qty'] * df_combined1['item_landed_price']

df_combined1

C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\591177248.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined1['Dispatched qty'].fillna(0, inplace=True)
C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\591177248.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

,Style ID,Wh_ID,Open PO,Dispatched qty,Dispatch Date,BU,style_id,item_landed_price,Remarks,Dispatch Value
0,29204096,36,176,0.0,,Personal Care,29204096,216.60,Brand Yet To Revert,0.00
1,22844406,36,0,0.0,,Personal Care,22844406,236.48,No Open Po,0.00
2,22292788,36,48,0.0,,Personal Care,22292788,147.26,Brand Yet To Revert,0.00
3,22292786,36,48,0.0,,Personal Care,22292786,149.40,Brand Yet To Revert,0.00
4,22292780,36,96,0.0,,Personal Care,22292780,149.40,Supply Issue,0.00
...,...,...,...,...,...,...,...,...,...,...
27129,23540690,81,30,30.0,20/03/25,Personal Care,23540690,126.40,Slotted,3792.00
27130,23540688,81,720,192.0,20/03/25,Personal Care,23540688,75.05,Slotted,14409.60
27131,23540686,81,32,32.0,20/03/25,Personal Care,23540686,94.80,Slotted,3033.60
27132,23540684,81,25,24.0,20/03/25,Personal Care,23540684,86.90,Slotted,2085.60


In [26]:
df_combined1['Remarks'] = df_combined1['Remarks'].replace('', 'Others')

In [27]:
df_combined2 = df_combined1[["BU", "Style ID", "Wh_ID","Dispatch Value", "Dispatched qty", "Dispatch Date", "Remarks"]]
df_combined2

,BU,Style ID,Wh_ID,Dispatch Value,Dispatched qty,Dispatch Date,Remarks
0,Personal Care,29204096,36,0.00,0.0,,Brand Yet To Revert
1,Personal Care,22844406,36,0.00,0.0,,No Open Po
2,Personal Care,22292788,36,0.00,0.0,,Brand Yet To Revert
3,Personal Care,22292786,36,0.00,0.0,,Brand Yet To Revert
4,Personal Care,22292780,36,0.00,0.0,,Supply Issue
...,...,...,...,...,...,...,...
27129,Personal Care,23540690,81,3792.00,30.0,20/03/25,Slotted
27130,Personal Care,23540688,81,14409.60,192.0,20/03/25,Slotted
27131,Personal Care,23540686,81,3033.60,32.0,20/03/25,Slotted
27132,Personal Care,23540684,81,2085.60,24.0,20/03/25,Slotted


In [28]:
## Data format dd/mm/yy
df_combined2['Dispatch Date'] = pd.to_datetime(df_combined2['Dispatch Date'], format='%d/%m/%y', errors='coerce')
df_combined2['Dispatch Date'] = df_combined2['Dispatch Date'].dt.strftime('%d/%m/%y').fillna('')
df_combined2

C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\3520530968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined2['Dispatch Date'] = pd.to_datetime(df_combined2['Dispatch Date'], format='%d/%m/%y', errors='coerce')
C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\3520530968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined2['Dispatch Date'] = df_combined2['Dispatch Date'].dt.strftime('%d/%m/%y').fillna('')


,BU,Style ID,Wh_ID,Dispatch Value,Dispatched qty,Dispatch Date,Remarks
0,Personal Care,29204096,36,0.00,0.0,,Brand Yet To Revert
1,Personal Care,22844406,36,0.00,0.0,,No Open Po
2,Personal Care,22292788,36,0.00,0.0,,Brand Yet To Revert
3,Personal Care,22292786,36,0.00,0.0,,Brand Yet To Revert
4,Personal Care,22292780,36,0.00,0.0,,Supply Issue
...,...,...,...,...,...,...,...
27129,Personal Care,23540690,81,3792.00,30.0,20/03/25,Slotted
27130,Personal Care,23540688,81,14409.60,192.0,20/03/25,Slotted
27131,Personal Care,23540686,81,3033.60,32.0,20/03/25,Slotted
27132,Personal Care,23540684,81,2085.60,24.0,20/03/25,Slotted


In [29]:
import datetime
from datetime import date, timedelta
date_today_new = datetime.datetime.now().strftime("%Y-%m-%d")
date_today_new

'2025-03-20'

In [30]:
df_combined2['Last_updated_on'] = date_today_new
df_combined2

C:\Users\karan.singhal\AppData\Local\Temp\ipykernel_7564\3670325003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined2['Last_updated_on'] = date_today_new


,BU,Style ID,Wh_ID,Dispatch Value,Dispatched qty,Dispatch Date,Remarks,Last_updated_on
0,Personal Care,29204096,36,0.00,0.0,,Brand Yet To Revert,2025-03-20
1,Personal Care,22844406,36,0.00,0.0,,No Open Po,2025-03-20
2,Personal Care,22292788,36,0.00,0.0,,Brand Yet To Revert,2025-03-20
3,Personal Care,22292786,36,0.00,0.0,,Brand Yet To Revert,2025-03-20
4,Personal Care,22292780,36,0.00,0.0,,Supply Issue,2025-03-20
...,...,...,...,...,...,...,...,...
27129,Personal Care,23540690,81,3792.00,30.0,20/03/25,Slotted,2025-03-20
27130,Personal Care,23540688,81,14409.60,192.0,20/03/25,Slotted,2025-03-20
27131,Personal Care,23540686,81,3033.60,32.0,20/03/25,Slotted,2025-03-20
27132,Personal Care,23540684,81,2085.60,24.0,20/03/25,Slotted,2025-03-20


In [31]:
df_combined2.columns

Index(['BU', 'Style ID', 'Wh_ID', 'Dispatch Value', 'Dispatched qty',
       'Dispatch Date', 'Remarks', 'Last_updated_on'],
      dtype='object')

In [32]:
df_combined3 = df_combined2[['BU', 'Style ID', 'Wh_ID', 'Dispatch Value',
       'Dispatch Date', 'Remarks','Dispatched qty', 'Last_updated_on']]

In [33]:
# Convert the dispatch_date column from string to datetime using the correct format
df_combined3['Dispatch Date'] = pd.to_datetime(df_combined3['Dispatch Date'], format='%d/%m/%y', errors='coerce')

# Extract day, month, and year from the datetime column
df_combined3['day'] = df_combined3['Dispatch Date'].dt.day
df_combined3['month'] = df_combined3['Dispatch Date'].dt.month
df_combined3['year'] = df_combined3['Dispatch Date'].dt.year

# Replace NaT (not a time) with empty or placeholder values
df_combined3.fillna({'day': '', 'month': '', 'year': ''}, inplace=True)

df_combined3

,BU,Style ID,Wh_ID,Dispatch Value,Dispatch Date,Remarks,Dispatched qty,Last_updated_on,day,month,year
0,Personal Care,29204096,36,0.00,NaT,Brand Yet To Revert,0.0,2025-03-20,,,
1,Personal Care,22844406,36,0.00,NaT,No Open Po,0.0,2025-03-20,,,
2,Personal Care,22292788,36,0.00,NaT,Brand Yet To Revert,0.0,2025-03-20,,,
3,Personal Care,22292786,36,0.00,NaT,Brand Yet To Revert,0.0,2025-03-20,,,
4,Personal Care,22292780,36,0.00,NaT,Supply Issue,0.0,2025-03-20,,,
...,...,...,...,...,...,...,...,...,...,...,...
27129,Personal Care,23540690,81,3792.00,2025-03-20,Slotted,30.0,2025-03-20,20.0,3.0,2025.0
27130,Personal Care,23540688,81,14409.60,2025-03-20,Slotted,192.0,2025-03-20,20.0,3.0,2025.0
27131,Personal Care,23540686,81,3033.60,2025-03-20,Slotted,32.0,2025-03-20,20.0,3.0,2025.0
27132,Personal Care,23540684,81,2085.60,2025-03-20,Slotted,24.0,2025-03-20,20.0,3.0,2025.0


In [34]:
df_combined4 = df_combined3[['BU', 'Style ID', 'Wh_ID', 'Dispatch Value', 'day', 'month', 'year',
       'Dispatch Date', 'Remarks','Dispatched qty', 'Last_updated_on']]
df_combined4

,BU,Style ID,Wh_ID,Dispatch Value,day,month,year,Dispatch Date,Remarks,Dispatched qty,Last_updated_on
0,Personal Care,29204096,36,0.00,,,,NaT,Brand Yet To Revert,0.0,2025-03-20
1,Personal Care,22844406,36,0.00,,,,NaT,No Open Po,0.0,2025-03-20
2,Personal Care,22292788,36,0.00,,,,NaT,Brand Yet To Revert,0.0,2025-03-20
3,Personal Care,22292786,36,0.00,,,,NaT,Brand Yet To Revert,0.0,2025-03-20
4,Personal Care,22292780,36,0.00,,,,NaT,Supply Issue,0.0,2025-03-20
...,...,...,...,...,...,...,...,...,...,...,...
27129,Personal Care,23540690,81,3792.00,20.0,3.0,2025.0,2025-03-20,Slotted,30.0,2025-03-20
27130,Personal Care,23540688,81,14409.60,20.0,3.0,2025.0,2025-03-20,Slotted,192.0,2025-03-20
27131,Personal Care,23540686,81,3033.60,20.0,3.0,2025.0,2025-03-20,Slotted,32.0,2025-03-20
27132,Personal Care,23540684,81,2085.60,20.0,3.0,2025.0,2025-03-20,Slotted,24.0,2025-03-20


In [35]:
df_combined2.to_csv(f'C:/Users/karan.singhal/Desktop/Bio_team/po_update_{date_today_new}.csv', index= False, header = False)

In [35]:
from azure.storage.blob import BlobClient
output_path = "C:/Users/karan.singhal/Desktop/Bio_team/"

blob = BlobClient(
    account_url="https://analyticsfereportci.blob.core.windows.net",
    container_name="mp-ops",  # Container name
    blob_name='bio_team/'+'po_update_'+ date_today_new + '/' + 'po_update_'+ date_today_new + '.csv',  # File path within the container
    credential="?sv=2020-02-10&st=2024-08-30T10%3A50%3A28Z&se=2028-08-31T10%3A50%3A00Z&sr=c&sp=racwdlmeop&sig=Wf9qbpnPOgNnOrRKMZXtjb7WAPaZfwP7aa8qxqxyXN4%3D"
)

with open(output_path + 'po_update_'+ date_today_new + '.csv', "rb") as data:
    blob.upload_blob(data, overwrite=True)

In [ ]:
cur.execute(""" drop table if exists hive.dev_7.po_details""")

cur.execute(""" 

CREATE TABLE hive.dev_7.po_details (
BU varchar,
Style_ID varchar,
whi_id varchar,
Dispatch_value varchar,
Dispatched_qty varchar,
Dispatch_Date varchar,
Remarks varchar,
Last_updated_on varchar

)

WITH (format='CSV', EXTERNAL_location ='abfss://mp-ops@analyticsfereportci.dfs.core.windows.net/bio_team/po_update_""" + date_today_new + """/')""")

In [ ]:
cur.execute(""" drop table if exists delta_prod.dev_1.po_details """)

cur.execute(""" create table delta_prod.dev_1.po_details as
                select * from delta_prod.dev_30.po_details
                union 
                select bu,
                CASE
                    WHEN CAST(style_id AS VARCHAR) LIKE '%.0' 
                    THEN CAST(REPLACE(CAST(style_id AS VARCHAR), '.0', '') AS BIGINT)
                    ELSE CAST(style_id AS BIGINT)
                END AS style_id,
                CASE
                    WHEN CAST(whi_id AS VARCHAR) LIKE '%.0' 
                    THEN CAST(REPLACE(CAST(whi_id AS VARCHAR), '.0', '') AS BIGINT)
                    ELSE CAST(whi_id AS BIGINT)
                END AS whi_id,
                CASE
                    WHEN TRIM(CAST(dispatch_value AS VARCHAR)) LIKE '%,%'
                    THEN CAST(REPLACE(TRIM(CAST(dispatch_value AS VARCHAR)), ',', '') AS DECIMAL(38,2))
                    ELSE CAST(dispatch_value AS DECIMAL(38,2))
                END AS dispatch_value,
                CASE
                    WHEN CAST(dispatched_qty AS VARCHAR) LIKE '%.0' 
                    THEN CAST(REPLACE(CAST(dispatched_qty AS VARCHAR), '.0', '') AS BIGINT)
                    ELSE CAST(dispatched_qty AS BIGINT)
                END AS dispatched_qty,
                dispatch_date,
                remarks,
                CASE
                    WHEN substr(trim(last_updated_on), 3, 1) = '-'
                    THEN CAST(DATE_PARSE(substr(trim(last_updated_on), 1, 10), '%d-%m-%Y') AS DATE)
                    WHEN substr(trim(last_updated_on), 5, 1) = '-'
                    THEN CAST(DATE_PARSE(substr(trim(last_updated_on), 1, 10), '%Y-%m-%d') AS DATE)
                    ELSE cast(last_updated_on as date)
                END AS last_updated_on from hive.dev_7.po_details
       """)

In [ ]:
cur.execute(""" drop table if exists delta_prod.dev_30.po_details """)

cur.execute(""" create table delta_prod.dev_30.po_details as
                SELECT * FROM delta_prod.dev_1.po_details
                
       """)

In [ ]:
df_combined4 = df_combined4.drop_duplicates()

In [ ]:
write_to_googlesheet(df_combined4, 'https://docs.google.com/spreadsheets/d/1YtZQ7PDNUHIXM_v3O5FRSCj6rD8C1a8O0Rf6RhFE5Jc/edit?usp=sharing', 'Slotting')

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Sender and receiver email addresses
fromaddr = "karan.singhal@myntra.com"
password = "ziis qwbo uicl bgia"
toaddr = ['harshal.meshram@myntra.com', 'salil.parashar@myntra.com', 'gaurav.rane@myntra.com', 'basavaraj.p1@myntra.com', 'mannem.teja@myntra.com',
         'shobendra.jain1@myntra.com', 'mayank.jain4@myntra.com', 'saravanan.palani@myntra.com', 'deepikanaik.vc@myntra.com']

# Create the email message
msg = MIMEMultipart()
msg['From'] = fromaddr
msg['To'] = ", ".join(toaddr)
msg['Subject'] = " BPC Availability | Tracker|" + " " + date_today_new  # Adding current timestamp

# HTML body content for the email
h1 = """
<html>
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <style type="text/css">
        table {
            font-family: arial, sans-serif;
            border-collapse: collapse;
            width: 55%;
        }
        th {
            background-color: #1e356f;
            border: 2px solid BLACK;
            text-align: center;
            padding: 4px;
            color: #D3D3D3;
        }
        td {
            border: 2px solid BLACK;
            text-align: center;
            padding: 4px;
        }
        tr {
            background-color: #F5F5F5;
        }
    </style>
</head>
<body>
    <p>Hi Team,</p>
    <p>The BPC Availability Tracker - Speed Stock Sufficiency is updated.</p>
    <p>Use the link to access the data: <a href="https://docs.google.com/spreadsheets/d/1YtZQ7PDNUHIXM_v3O5FRSCj6rD8C1a8O0Rf6RhFE5Jc/edit?usp=sharing" target="_blank">Click here to access the data</a></p>
    <p>Best Regards, <br> Karan Singhal</p>
</body>
</html>
"""

# Attach the HTML body content to the email
msg.attach(MIMEText(h1, 'html'))

# Send the email using SMTP
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, password)
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
print("Mail sent")
server.quit()  # Close the connection